In [1]:
import osmium as osm
import pandas as pd
import datetime
import pytz

In [2]:
class TimelineHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.elemtimeline = []

    def element(self, e, elem_type):
        self.elemtimeline.append([elem_type,
                                  e.id,
                                  e.version,
                                  e.visible,
                                  pd.Timestamp(e.timestamp),
                                  e.uid,
                                  e.changeset,
                                  len(e.tags)])

    def node(self, n):
        self.element(n, "node")

    def way(self, w):
        self.element(w, "way")

    def relation(self, r):
        self.element(r, "relation")

In [3]:
tlhandler = TimelineHandler()
tlhandler.apply_file("data/ottgat.osh.pbf")
colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid', 'chgset', 'ntags']
elements = pd.DataFrame(tlhandler.elemtimeline, columns=colnames)
elements = elements.sort_values(by=['type', 'id', 'ts'])

#elements.to_csv("output/elements.csv", date_format='%Y-%m-%d %H:%M:%S')
#elements.to_csv("output/elements.csv")
elements.to_csv("output/elements.csv", date_format='%Y-%m-%d')